In [6]:
import pandas as pd
import numpy as np
import yfinance as yf

In [ ]:
# Step 1: Data Acquisition

In [7]:
ticker = "MSFT"  # Replace with the ticker symbol of the desired Indian equity
start_date = "2013-07-07"
end_date = "2023-07-06"


In [3]:
# Retrieve historical price data using yfinance

In [8]:
data = yf.download(ticker, start=start_date, end=end_date)
df = data[['Open', 'High', 'Low', 'Close']].copy()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2013-07-08,34.349998,34.590000,33.980000,34.330002
2013-07-09,34.580002,34.599998,34.139999,34.349998
2013-07-10,34.340000,34.810001,34.320000,34.700001
2013-07-11,35.000000,35.770000,34.900002,35.689999
2013-07-12,35.580002,35.730000,35.279999,35.669998
...,...,...,...,...
2023-06-28,334.660004,337.980011,333.809998,335.850006
2023-06-29,334.709991,336.109985,332.619995,335.049988
2023-06-30,337.750000,342.730011,337.200012,340.540009


In [25]:
rsi_period = 14

In [1]:
# Calculate MACD
def calculate_macd(df, short_period=12, long_period=26, signal_period=9):
    df['EMA_short'] = df['Close'].ewm(span=short_period, adjust=False).mean()
    df['EMA_long'] = df['Close'].ewm(span=long_period, adjust=False).mean()
    df['MACD'] = df['EMA_short'] - df['EMA_long']
    df['Signal'] = df['MACD'].ewm(span=signal_period, adjust=False).mean()

calculate_macd(df)

NameError: name 'df' is not defined

In [28]:
# Calculate average gain and average loss
avg_gain = gain.rolling(window=rsi_period).mean()
avg_loss = loss.rolling(window=rsi_period).mean()

In [29]:
# Calculate relative strength (RS)
rs = avg_gain / avg_loss

# Calculate RSI
rsi = 100 - (100 / (1 + rs))

In [30]:
df['RSI'] = rsi

In [44]:
# Step 4: Identification of Chart Pattern
def is_double_top(highs):
    if len(highs) < 3:
        return False
    first_peak = np.argmax(highs[:-1])
    if first_peak == 0:
        return False
    second_peak = np.argmax(highs[first_peak+1:]) + first_peak + 1
    if second_peak == first_peak or second_peak >= len(highs) - 1:
        return False
    if np.min(highs[first_peak+1:second_peak+1]) < highs[first_peak]:
        return False
    if highs[second_peak] >= highs[first_peak]:
        return True
    return False



In [45]:
# Step 5: Entry and Exit Rules
rsi_threshold = 30
exit_threshold = 50

position = 0
entry_price = 0.0
exit_price = 0.0
trades = []

for i, row in df.iterrows():
    if position == 0:
        if row['RSI'] < rsi_threshold and row['DoubleTop']:
            position = 1
            entry_price = row['Close']
    elif position == 1:
        if row['RSI'] >= exit_threshold or row['Close'] < entry_price:
            position = 0
            exit_price = row['Close']
            trades.append(exit_price / entry_price - 1)

# Step 6: Backtesting and Evaluation
cumulative_returns = np.prod(1 + trades) - 1
max_drawdown = np.max(np.cumprod(1 + trades) - 1 - np.maximum.accumulate(np.cumprod(1 + trades) - 1))
returns_std = np.std(trades)
sharpe_ratio = np.sqrt(len(trades)) * np.mean(trades) / returns_std

print("Cumulative Returns: {:.2%}".format(cumulative_returns))
print("Maximum Drawdown: {:.2%}".format(max_drawdown))
print("Sharpe Ratio: {:.2f}".format(sharpe_ratio))

KeyError: 'RSI'